In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from mrcnn.config import Config
#import utils
from mrcnn import model as modellib,utils
from mrcnn import visualize
import yaml
from mrcnn.model import log
from PIL import Image
 
 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Root directory of the project
ROOT_DIR = os.getcwd()
ROOT_DIR += '/Mask_RCNN/' 
#ROOT_DIR = os.path.abspath("../")
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

iter_num=0

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
print(COCO_MODEL_PATH,MODEL_DIR) 
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

Using TensorFlow backend.


/home/zkl/zklcode/code/helmet/123123/Mask_RCNN/Mask_RCNN/mask_rcnn_coco.h5 /home/zkl/zklcode/code/helmet/123123/Mask_RCNN/Mask_RCNN/logs


FileNotFoundError: [Errno 2] No such file or directory: '/home/zkl/zklcode/code/helmet/123123/Mask_RCNN/Mask_RCNN/mask_rcnn_coco.h5'

In [6]:
class ShapesConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "shapes"
 
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
 
    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 3 shapes
 
    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 320
    IMAGE_MAX_DIM = 384
 
    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8 * 6, 16 * 6, 32 * 6, 64 * 6, 128 * 6)  # anchor side in pixels
 
    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 100
 
    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100
 
    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 50
 
 
config = ShapesConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  384
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  320
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [384 384   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

In [7]:
class DrugDataset(utils.Dataset):
    # 得到该图中有多少个实例（物体）
    def get_obj_index(self, image):
        n = np.max(image)
        return n
 
    # 解析labelme中得到的yaml文件，从而得到mask每一层对应的实例标签
    def from_yaml_get_class(self, image_id):
        info = self.image_info[image_id]
        with open(info['yaml_path']) as f:
            temp = yaml.load(f.read())
            labels = temp['label_names']
            del labels[0]
        return labels
 
    # 重新写draw_mask
    def draw_mask(self, num_obj, mask, image,image_id):
        #print("draw_mask-->",image_id)
        #print("self.image_info",self.image_info)
        info = self.image_info[image_id]
        #print("info-->",info)
        #print("info[width]----->",info['width'],"-info[height]--->",info['height'])
        for index in range(num_obj):
            for i in range(info['width']):
                for j in range(info['height']):
                    #print("image_id-->",image_id,"-i--->",i,"-j--->",j)
                    #print("info[width]----->",info['width'],"-info[height]--->",info['height'])
                    at_pixel = image.getpixel((i, j))
                    if at_pixel == index + 1:
                        mask[j, i, index] = 1
        return mask
 
    # 重新写load_shapes，里面包含自己的自己的类别
    # 并在self.image_info信息中添加了path、mask_path 、yaml_path
    # yaml_pathdataset_root_path = "/tongue_dateset/"
    # img_floder = dataset_root_path + "rgb"
    # mask_floder = dataset_root_path + "mask"
    # dataset_root_path = "/tongue_dateset/"
    def load_shapes(self, count, img_floder, mask_floder, imglist, dataset_root_path):
        """Generate the requested number of synthetic images.
        count: number of images to generate.
        height, width: the size of the generated images.
        """
        # Add classes
        self.add_class("shapes", 1, "helmat")
        #self.add_class("shapes", 2, "leg")
        #self.add_class("shapes", 3, "well")
 
        for i in range(count):
            # 获取图片宽和高
 
            filestr = imglist[i].split(".")[0]
            #print(imglist[i],"-->",cv_img.shape[1],"--->",cv_img.shape[0])
            #print("id-->", i, " imglist[", i, "]-->", imglist[i],"filestr-->",filestr)
            # filestr = filestr.split("_")[1]
            mask_path = mask_floder + "/" + filestr + ".png"
            yaml_path = dataset_root_path + "labelme_json/" + filestr + "_json/info.yaml"
            print(dataset_root_path + "labelme_json/" + filestr + "_json/img.png")
            cv_img = cv2.imread(dataset_root_path + "labelme_json/" + filestr + "_json/img.png")
 
            self.add_image("shapes", image_id=i, path=img_floder + "/" + imglist[i],
                           width=cv_img.shape[1], height=cv_img.shape[0], mask_path=mask_path, yaml_path=yaml_path)
 
    # 重写load_mask
    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        global iter_num
        print("image_id",image_id)
        info = self.image_info[image_id]
        count = 1  # number of object
        img = Image.open(info['mask_path'])
        num_obj = self.get_obj_index(img)
        mask = np.zeros([info['height'], info['width'], num_obj], dtype=np.uint8)
        mask = self.draw_mask(num_obj, mask, img,image_id)
        occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
        for i in range(count - 2, -1, -1):
            mask[:, :, i] = mask[:, :, i] * occlusion
 
            occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i]))
        labels = []
        labels = self.from_yaml_get_class(image_id)
        labels_form = []
        for i in range(len(labels)):
            if labels[i].find("helmat") != -1:
                # print "car"
                labels_form.append("helmat")
            # elif labels[i].find("leg") != -1:
            #     # print "leg"
            #     labels_form.append("leg")
            # elif labels[i].find("well") != -1:
            #     # print "well"
            #     labels_form.append("well")
        class_ids = np.array([self.class_names.index(s) for s in labels_form])
        return mask, class_ids.astype(np.int32)
 
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
 
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size * cols, size * rows))
    return ax
ROOT_DIR = os.path.abspath("../")
dataset_root_path=ROOT_DIR +"/helmat/helmat/dataset/"
dataset_root_path 
#基础设置
# ROOT_DIR = os.path.abspath("../")
dataset_root_path="../helmat/helmat/dataset/"
img_floder = dataset_root_path + "pic"
mask_floder = dataset_root_path + "cv2_mask"
#yaml_floder = dataset_root_path
imglist = os.listdir(img_floder)
count = len(imglist)
 
#train与val数据集准备
dataset_train = DrugDataset()
dataset_train.load_shapes(count, img_floder, mask_floder, imglist,dataset_root_path)
dataset_train.prepare()

dataset_val = DrugDataset()
dataset_val.load_shapes(10, img_floder, mask_floder, imglist,dataset_root_path)
dataset_val.prepare()

../helmat/helmat/dataset/labelme_json/1_json/img.png
../helmat/helmat/dataset/labelme_json/00106_json/img.png
../helmat/helmat/dataset/labelme_json/001_json/img.png
../helmat/helmat/dataset/labelme_json/10_json/img.png
../helmat/helmat/dataset/labelme_json/100_json/img.png
../helmat/helmat/dataset/labelme_json/00108_json/img.png
../helmat/helmat/dataset/labelme_json/102_json/img.png
../helmat/helmat/dataset/labelme_json/00101_json/img.png
../helmat/helmat/dataset/labelme_json/0011_json/img.png
../helmat/helmat/dataset/labelme_json/00107_json/img.png
../helmat/helmat/dataset/labelme_json/00102_json/img.png
../helmat/helmat/dataset/labelme_json/00104_json/img.png
../helmat/helmat/dataset/labelme_json/105_json/img.png
../helmat/helmat/dataset/labelme_json/00100_json/img.png
../helmat/helmat/dataset/labelme_json/00103_json/img.png
../helmat/helmat/dataset/labelme_json/109_json/img.png
../helmat/helmat/dataset/labelme_json/0010_json/img.png
../helmat/helmat/dataset/labelme_json/107_json/img

In [2]:
ROOT_DIR = os.path.abspath("../")
dataset_root_path=ROOT_DIR +"/helmat/helmat/dataset/"
dataset_root_path

'/home/zkl/zklcode/code/helmet/123123/helmat/helmat/dataset/'

In [3]:
#基础设置
# ROOT_DIR = os.path.abspath("../")
dataset_root_path="../helmat/helmat/dataset/"
img_floder = dataset_root_path + "pic"
mask_floder = dataset_root_path + "cv2_mask"
#yaml_floder = dataset_root_path
imglist = os.listdir(img_floder)
count = len(imglist)
 
#train与val数据集准备
dataset_train = DrugDataset()
dataset_train.load_shapes(count, img_floder, mask_floder, imglist,dataset_root_path)
dataset_train.prepare()

NameError: name 'DrugDataset' is not defined

In [28]:
dataset_val = DrugDataset()
dataset_val.load_shapes(10, img_floder, mask_floder, imglist,dataset_root_path)
dataset_val.prepare()

../helmat/helmat/dataset/labelme_json/1_json/img.png
../helmat/helmat/dataset/labelme_json/00106_json/img.png
../helmat/helmat/dataset/labelme_json/001_json/img.png
../helmat/helmat/dataset/labelme_json/10_json/img.png
../helmat/helmat/dataset/labelme_json/100_json/img.png
../helmat/helmat/dataset/labelme_json/00108_json/img.png
../helmat/helmat/dataset/labelme_json/102_json/img.png
../helmat/helmat/dataset/labelme_json/00101_json/img.png
../helmat/helmat/dataset/labelme_json/0011_json/img.png
../helmat/helmat/dataset/labelme_json/00107_json/img.png


In [8]:
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)
print(model,COCO_MODEL_PATH)
model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])

<mrcnn.model.MaskRCNN object at 0x7f6c86a4e8d0> /home/zkl/zklcode/code/helmet/123123/Mask_RCNN/Mask_RCNN/mask_rcnn_coco.h5


OSError: Unable to open file (unable to open file: name = '/home/zkl/zklcode/code/helmet/123123/Mask_RCNN/Mask_RCNN/mask_rcnn_coco.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [10]:
COCO_MODEL_PATH ='/home/zkl/zklcode/code/helmet/123123/Mask_RCNN/mask_rcnn_coco.h5'

print(COCO_MODEL_PATH)
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last
 
if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    # print(COCO_MODEL_PATH)
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)
 


/home/zkl/zklcode/code/helmet/123123/Mask_RCNN/mask_rcnn_coco.h5


In [11]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=10,
            layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: /home/zkl/zklcode/code/helmet/123123/Mask_RCNN/Mask_RCNN/logs/shapes20181219T1210/mask_rcnn_shapes_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4

/home/zkl/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/zkl/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/10
image_id 23
image_id 23
image_id 23
image_id 23
image_id 23
image_id 23
image_id 23
image_id 23
image_id 5
image_id 5
image_id 5
image_id 5
image_id 5
image_id 5
image_id 5
image_id 5
image_id 14
image_id 14
image_id 14
image_id 14
image_id 14
image_id 14
image_id 14
image_id 14
image_id 7
image_id 7
image_id 7
image_id 7
image_id 7
image_id 0
image_id 0
image_id 7
image_id 7
image_id 0
image_id 7
image_id 0
image_id 0
image_id 0
image_id 0
image_id 0
image_id 11
image_id 11
image_id 11
image_id 11
image_id 11
image_id 4
image_id 11
image_id 4
image_id 4
image_id 11
image_id 4
image_id 4
image_id 11
image_id 4
image_id 1
image_id 1
image_id 1
image_id 4
image_id 1
image_id 4
image_id 1
image_id 1
image_id 1
image_id 1
image_id 20
image_id 3
image_id 3
image_id 3
image_id 20
image_id 20
image_id 20
image_id 3
image_id 20
image_id 3
image_id 3
image_id 20
image_id 12
image_id 20
image_id 20
image_id 3
image_id 3
image_id 3
image_id 12
image_id 12
image_id 12
image_id 3
image_i

 67/100 [===================>..........] - ETA: 20s - loss: 1.5962 - rpn_class_loss: 0.0307 - rpn_bbox_loss: 0.5180 - mrcnn_class_loss: 0.1347 - mrcnn_bbox_loss: 0.6230 - mrcnn_mask_loss: 0.2898image_id 10


 99/100 [============================>.] - ETA: 0s - loss: 1.5765 - rpn_class_loss: 0.0306 - rpn_bbox_loss: 0.6021 - mrcnn_class_loss: 0.1147 - mrcnn_bbox_loss: 0.5844 - mrcnn_mask_loss: 0.2448image_id 22
image_id 7
image_id 7
image_id 9
image_id 7
image_id 9
image_id 9
image_id 9
image_id 9
image_id 9
image_id 5
image_id 9
image_id 5
image_id 5
image_id 9
image_id 5
image_id 5
image_id 5
image_id 5
image_id 5
image_id 4
image_id 4
image_id 4
image_id 4
image_id 0
image_id 0
image_id 4
image_id 0
image_id 0
image_id 4
image_id 4
image_id 4
image_id 2
image_id 2
image_id 0
image_id 2
image_id 0
image_id 2
image_id 0
image_id 0
image_id 2
image_id 1
image_id 1
image_id 2
image_id 1
image_id 2
image_id 1
image_id 2
image_id 1
image_id 6
image_id 6
100/100 [==============================] - 56s 559ms/step - loss: 1.5677 - rpn_class_loss: 0.0303 - rpn_bbox_loss: 0.5972 - mrcnn_class_loss: 0.1138 - mrcnn_bbox_loss: 0.5828 - mrcnn_mask_loss: 0.2436 - val_loss: 1.5530 - val_rpn_class_loss: 0.0

...

 69/100 [===================>..........] - ETA: 6s - loss: 0.1187 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0265 - mrcnn_class_loss: 0.0205 - mrcnn_bbox_loss: 0.0215 - mrcnn_mask_loss: 0.0491image_id 17


 99/100 [============================>.] - ETA: 0s - loss: 0.1448 - rpn_class_loss: 0.0015 - rpn_bbox_loss: 0.0342 - mrcnn_class_loss: 0.0232 - mrcnn_bbox_loss: 0.0293 - mrcnn_mask_loss: 0.0566image_id 11
image_id 8
image_id 8
image_id 2
image_id 2
image_id 1
image_id 5
image_id 4
image_id 4
image_id 2
image_id 2
image_id 7
image_id 7
image_id 1
image_id 1
image_id 0
image_id 1
image_id 1
image_id 6
image_id 5
image_id 5
image_id 9
image_id 0
image_id 3
image_id 0
image_id 9
image_id 0
image_id 0
image_id 4
image_id 3
image_id 3
image_id 3
image_id 1
image_id 3
image_id 6
image_id 6
image_id 9
image_id 9
image_id 6
image_id 9
image_id 9
image_id 4
image_id 7
image_id 4
image_id 4
image_id 1
image_id 7
image_id 1
100/100 [==============================] - 25s 255ms/step - loss: 0.1443 - rpn_class_loss: 0.0015 - rpn_bbox_loss: 0.0339 - mrcnn_class_loss: 0.0231 - mrcnn_bbox_loss: 0.0292 - mrcnn_mask_loss: 0.0566 - val_loss: 0.2694 - val_rpn_class_loss: 0.0034 - val_rpn_bbox_loss: 0.0582 -

 44/100 [============>.................] - ETA: 11s - loss: 0.2331 - rpn_class_loss: 0.0044 - rpn_bbox_loss: 0.0331 - mrcnn_class_loss: 0.0505 - mrcnn_bbox_loss: 0.0722 - mrcnn_mask_loss: 0.0730image_id 2


 84/100 [========================>.....] - ETA: 3s - loss: 0.1854 - rpn_class_loss: 0.0035 - rpn_bbox_loss: 0.0333 - mrcnn_class_loss: 0.0384 - mrcnn_bbox_loss: 0.0491 - mrcnn_mask_loss: 0.0611image_id 7


 99/100 [============================>.] - ETA: 0s - loss: 0.1726 - rpn_class_loss: 0.0031 - rpn_bbox_loss: 0.0332 - mrcnn_class_loss: 0.0341 - mrcnn_bbox_loss: 0.0434 - mrcnn_mask_loss: 0.0588image_id 17
image_id 4
image_id 7
image_id 7
image_id 4
image_id 5
image_id 6
image_id 6
image_id 3
image_id 7
image_id 7
image_id 5
image_id 5
image_id 7
image_id 7
image_id 6
image_id 5
image_id 5
image_id 6
image_id 9
image_id 5
image_id 6
image_id 5
image_id 9
image_id 9
image_id 9
image_id 6
image_id 9
image_id 6
image_id 9
image_id 4
image_id 0
image_id 4
image_id 4
image_id 9
image_id 1
image_id 8
image_id 9
image_id 3
image_id 1
image_id 3
image_id 9
image_id 1
image_id 9
image_id 4
image_id 6
image_id 2
image_id 6
image_id 4
image_id 7
image_id 1
100/100 [==============================] - 25s 252ms/step - loss: 0.1716 - rpn_class_loss: 0.0031 - rpn_bbox_loss: 0.0330 - mrcnn_class_loss: 0.0340 - mrcnn_bbox_loss: 0.0431 - mrcnn_mask_loss: 0.0586 - val_loss: 0.2032 - val_rpn_class_loss: 0.0

In [12]:
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=3,
            layers="all")


Starting at epoch 10. LR=0.0001

Checkpoint Path: /home/zkl/zklcode/code/helmet/123123/Mask_RCNN/Mask_RCNN/logs/shapes20181219T1210/mask_rcnn_shapes_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a

image_id 5
image_id 5
image_id 5
image_id 5
image_id 5
image_id 5
image_id 5
image_id 5
